# Customer + Orders + Returns Analysis (Full)
All code from this chat consolidated into one notebook.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
CUSTOMERS_PATH = r"/mnt/data/customers.csv"
ORDERS_PATH = r"/mnt/data/orders.csv"
RETURNS_PATH = r"/mnt/data/returns.csv"

In [ ]:
customers = pd.read_csv(CUSTOMERS_PATH)
orders = pd.read_csv(ORDERS_PATH)
returns = pd.read_csv(RETURNS_PATH)
customers.head(), orders.head(), returns.head()

In [ ]:
def summarize_df(df: pd.DataFrame, descriptions: dict) -> pd.DataFrame:
    summary_rows = []
    for col in df.columns:
        series = df[col]
        row = {
            "Column": col,
            "Data Type": str(series.dtype),
            "Description": descriptions.get(col, ""),
            "Missing Values": int(series.isna().sum()),
            "Unique Values": int(series.nunique(dropna=True)),
        }
        if pd.api.types.is_numeric_dtype(series):
            row.update({
                "Min": float(series.min()),
                "Max": float(series.max()),
                "Mean": float(series.mean()),
                "Median": float(series.median()),
                "Std Dev": float(series.std()),
            })
        else:
            row.update({"Min": None, "Max": None, "Mean": None, "Median": None, "Std Dev": None})
        summary_rows.append(row)
    return pd.DataFrame(summary_rows)


In [ ]:
customer_desc = {
    "Customer ID": "Unique identifier for each customer",
    "Segment": "Customer segment (Consumer, Corporate, Home Office)",
    "Country": "Country where the customer is located",
    "City": "Customer city",
    "State": "Customer state",
    "Postal Code": "Customer postal/ZIP code",
    "Region": "Sales region",
}
orders_desc = {
    "Row ID": "Unique row identifier for each order line",
    "Order ID": "Unique order identifier",
    "Order Date": "Date the order was placed",
    "Ship Date": "Date the order was shipped",
    "Ship Mode": "Shipping method",
    "Customer ID": "Customer identifier",
    "Product ID": "Product identifier",
    "Category": "Product category",
    "Sub-Category": "Product sub-category",
    "Product Name": "Product name",
    "Sales": "Sales revenue for the order line",
    "Quantity": "Quantity ordered",
    "Discount": "Discount applied",
    "Profit": "Profit for the order line",
}
returns_desc = {
    "Returned": "Indicates whether the order was returned",
    "Order ID": "Identifier of the returned order",
}

customers_summary = summarize_df(customers, customer_desc)
orders_summary = summarize_df(orders, orders_desc)
returns_summary = summarize_df(returns, returns_desc)

customers_summary, orders_summary, returns_summary


In [ ]:
orders["Order Date"] = pd.to_datetime(orders["Order Date"], format="%m/%d/%y")
orders["Ship Date"] = pd.to_datetime(orders["Ship Date"], format="%m/%d/%y")
orders.dtypes


In [ ]:
unique_customers_customers = customers["Customer ID"].nunique()
unique_customers_orders = orders["Customer ID"].nunique()
unique_orders = orders["Order ID"].nunique()

orders_with_customer = orders.merge(customers[["Customer ID"]], on="Customer ID", how="left", indicator=True)
unmatched_order_rows = int((orders_with_customer["_merge"] != "both").sum())

returns_flag = returns.assign(Returned_Flag=1)[["Order ID", "Returned_Flag"]]
merged = orders.merge(customers, on="Customer ID", how="left").merge(returns_flag, on="Order ID", how="left")
merged["Returned_Flag"] = merged["Returned_Flag"].fillna(0).astype(int)

{
    "unique_customers_in_customers_table": unique_customers_customers,
    "unique_customers_in_orders_table": unique_customers_orders,
    "unique_orders": unique_orders,
    "unmatched_order_rows": unmatched_order_rows,
    "merged_shape": merged.shape
}


In [ ]:
orders["Month"] = orders["Order Date"].dt.to_period("M").dt.to_timestamp()

monthly_sales = orders.groupby("Month")["Sales"].sum().reset_index()

plt.figure()
plt.plot(monthly_sales["Month"], monthly_sales["Sales"])
plt.xlabel("Month")
plt.ylabel("Total Sales")
plt.title("Monthly Sales Over Time (Line Chart)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure()
orders.boxplot(column="Profit", by="Category")
plt.title("Profit Distribution by Category (Box Plot)")
plt.suptitle("")
plt.xlabel("Category")
plt.ylabel("Profit")
plt.tight_layout()
plt.show()


In [ ]:
monthly_sales_category = orders.groupby(["Month", "Category"])["Sales"].sum().reset_index()
pivot_all = monthly_sales_category.pivot(index="Month", columns="Category", values="Sales")

plt.figure(figsize=(14, 6))
for cat in pivot_all.columns:
    plt.plot(pivot_all.index, pivot_all[cat], label=cat)
plt.xlabel("Month")
plt.ylabel("Total Sales")
plt.title("Monthly Sales Over Time by Product Category (Line Chart)")
plt.legend(title="Category")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
start_date = pd.Timestamp("2016-07-01")
end_date = pd.Timestamp("2017-07-31")
filtered = orders[(orders["Month"] >= start_date) & (orders["Month"] <= end_date)]

monthly_sales_cat_window = filtered.groupby(["Month", "Category"])["Sales"].sum().reset_index()
pivot_window = monthly_sales_cat_window.pivot(index="Month", columns="Category", values="Sales")

plt.figure(figsize=(14, 6))
for cat in pivot_window.columns:
    plt.plot(pivot_window.index, pivot_window[cat], label=cat)
plt.xlabel("Month")
plt.ylabel("Total Sales")
plt.title("Monthly Sales by Product Category (July 2016 – July 2017)\n(Line Chart)")
plt.legend(title="Category")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
returns_only = returns.copy()
returns_only["Returned_Flag"] = 1
orders_ret = orders.merge(returns_only[["Order ID", "Returned_Flag"]], on="Order ID", how="left")
orders_ret["Returned_Flag"] = orders_ret["Returned_Flag"].fillna(0)

order_level = orders_ret.groupby(["Order ID", "Category"], as_index=False).agg(Returned=("Returned_Flag", "max"))
return_rate_by_category = order_level.groupby("Category")["Returned"].mean().reset_index(name="Return Rate")
return_rate_by_category


In [ ]:
tech = orders_ret[orders_ret["Category"] == "Technology"]
tech_order_level = tech.groupby(["Order ID", "Sub-Category"], as_index=False).agg(Returned=("Returned_Flag", "max"))
tech_return_rate = tech_order_level.groupby("Sub-Category")["Returned"].mean().reset_index(name="Return Rate").sort_values("Return Rate", ascending=False)
sanity_check = tech_order_level.groupby("Sub-Category").agg(Total_Orders=("Order ID", "nunique"), Returned_Orders=("Returned", "sum")).reset_index()

colors = ["#d62728", "#ff7f0e", "#2ca02c", "#1f77b4"]
plt.figure(figsize=(10, 6))
bars = plt.bar(tech_return_rate["Sub-Category"], tech_return_rate["Return Rate"], color=colors[:len(tech_return_rate)])
for bar, rate in zip(bars, tech_return_rate["Return Rate"]):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f"{rate:.1%}", ha="center", va="bottom", fontsize=10)
plt.xlabel("Technology Sub-Category")
plt.ylabel("Return Rate")
plt.title("Return Rate by Technology Sub-Category\n(With Percentage Labels)")
plt.xticks(rotation=30, ha="right")
plt.ylim(0, tech_return_rate["Return Rate"].max() * 1.3)
plt.tight_layout()
plt.show()

tech_return_rate, sanity_check


In [ ]:
orders_with_region = orders.merge(customers[["Customer ID", "Region"]], on="Customer ID", how="left")
sales_by_region = orders_with_region.groupby("Region")["Sales"].sum().reset_index().sort_values("Sales", ascending=False)

region_colors = {"West": "#1f77b4", "East": "#2ca02c", "Central": "#ff7f0e", "South": "#d62728"}
colors_region = [region_colors.get(r, "#7f7f7f") for r in sales_by_region["Region"]]

plt.figure(figsize=(8, 5))
bars = plt.bar(sales_by_region["Region"], sales_by_region["Sales"], color=colors_region)
for bar, value in zip(bars, sales_by_region["Sales"]):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f"${value/1_000_000:.2f}M", ha="center", va="bottom", fontsize=10)
plt.xlabel("Region")
plt.ylabel("Total Sales")
plt.title("Total Sales by Region\n(Color-Coded Bar Chart)")
plt.tight_layout()
plt.show()

sales_by_region


In [ ]:
monthly_sales_category = orders.groupby(["Month", "Category"])["Sales"].sum().reset_index()
pivot_all = monthly_sales_category.pivot(index="Month", columns="Category", values="Sales")

plt.figure(figsize=(14, 6))
for cat in pivot_all.columns:
    plt.plot(pivot_all.index, pivot_all[cat], label=cat)

years = pivot_all.index.year.unique()
for year in years:
    plt.axvspan(pd.Timestamp(f"{year}-10-01"), pd.Timestamp(f"{year}-12-31"), alpha=0.12)

plt.xlabel("Month")
plt.ylabel("Total Sales")
plt.title("Monthly Sales Over Time by Product Category\nShaded regions indicate Q4 (Oct–Dec) seasonality")
plt.legend(title="Category")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
orders_with_segment = orders.merge(customers[["Customer ID", "Segment"]], on="Customer ID", how="left")
profit_by_segment = orders_with_segment.groupby("Segment")["Profit"].sum().reset_index().sort_values("Profit", ascending=False)
consumer_profit = orders_with_segment.loc[orders_with_segment["Segment"] == "Consumer", "Profit"].sum()
profit_by_segment, consumer_profit


In [ ]:
orders_loc = orders.merge(customers[["Customer ID", "City", "State"]], on="Customer ID", how="left")
order_level_sales_loc = orders_loc.groupby(["Order ID", "City", "State"], as_index=False)["Sales"].sum()

aov_by_city = order_level_sales_loc.groupby("City")["Sales"].mean().reset_index(name="Average Order Value").sort_values("Average Order Value", ascending=False)
aov_by_state = order_level_sales_loc.groupby("State")["Sales"].mean().reset_index(name="Average Order Value").sort_values("Average Order Value", ascending=False)

saginaw_aov = order_level_sales_loc.loc[order_level_sales_loc["City"] == "Saginaw", "Sales"].mean()

aov_by_city.head(10), aov_by_state.head(10), saginaw_aov
